<a href="https://colab.research.google.com/github/Juanchi0207/SoporteInf6to/blob/main/RequestAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
import csv
import re

# Function to get airport data from API
def get_airport_data():
    url = "https://aviationweather.gov/api/data/airport"  # Replace with your API endpoint
    response = requests.get(url)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Function to parse the airport data
def parse_airport_data(data):
    airport_data = []
    airport_info = {}
    runways = []
    lines = data.split('\n')

    for line in lines:
        if line.startswith("AIRPORT INFO"):
            if airport_info:  # If there's already data collected, store it
                airport_info["Runways"] = runways
                airport_data.append(airport_info)
                airport_info = {}
                runways = []
        elif line.startswith("  Name:"):
            airport_info["Name"] = line.split(":")[1].strip()
        elif line.startswith("  IATA:"):
            airport_info["IATA"] = line.split(":")[1].strip()
        elif line.startswith("  FAA:"):
            airport_info["FAA"] = line.split(":")[1].strip()
        elif line.startswith("  Type:"):
            airport_info["Type"] = line.split(":")[1].strip()
        elif line.startswith("  Latitude:"):
            airport_info["Latitude"] = line.split(":")[1].strip()
        elif line.startswith("  Longitude:"):
            airport_info["Longitude"] = line.split(":")[1].strip()
        elif line.startswith("  Elevation:"):
            airport_info["Elevation"] = line.split(":")[1].strip()
        elif line.startswith("  Source:"):
            airport_info["Source"] = line.split(":")[1].strip()
        elif line.startswith("  State:"):
            airport_info["State"] = line.split(":")[1].strip()
        elif line.startswith("  Country:"):
            airport_info["Country"] = line.split(":")[1].strip()
        elif line.startswith("  Mag Declination:"):
            airport_info["Mag Declination"] = line.split(":")[1].strip()
        elif line.startswith("  Use:"):
            airport_info["Use"] = line.split(":")[1].strip()
        elif line.startswith("  Services:"):
            airport_info["Services"] = line.split(":")[1].strip()
        elif line.startswith("  Tower:"):
            airport_info["Tower"] = line.split(":")[1].strip()
        elif line.startswith("  Beacon:"):
            airport_info["Beacon"] = line.split(":")[1].strip()
        elif line.startswith("     Number:"):
            runway = {}
            runway["Number"] = line.split(":")[1].strip()
        elif line.startswith("     Length:"):
            runway["Length"] = line.split(":")[1].strip()
        elif line.startswith("     Type:"):
            runway["Type"] = line.split(":")[1].strip()
        elif line.startswith("     Runway:"):
            dimension_match = re.search(r'Dimension:\s*([0-9]+x[0-9]+)', line)
            surface_match = re.search(r'Surface:\s*([A-Z]+)', line)
            align_match = re.search(r'Align:\s*([0-9]+)', line)
            if dimension_match:
                runway["Dimension"] = dimension_match.group(1).strip()
            if surface_match:
                runway["Surface"] = surface_match.group(1).strip()
            if align_match:
                runway["Align"] = align_match.group(1).strip()
            runways.append(runway)

    # Add the last airport info
    if airport_info:
        airport_info["Runways"] = runways
        airport_data.append(airport_info)

    return airport_data

# Function to save data into a CSV file
def save_to_csv(airports, filename="airport_info.csv"):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Writing header
        writer.writerow([
            "Name", "IATA", "FAA", "Type", "Latitude", "Longitude", "Elevation",
            "Source", "State", "Country", "MagDeclination", "Use", "Services",
            "Tower", "Beacon", "RunwayNumber", "RunwayLength", "RunwayType",
            "RunwayDimension", "RunwaySurface", "RunwayAlign"
        ])

        # Writing airport data
        for airport in airports:
            runways = airport["Runways"]
            if runways:
                for runway in runways:
                    writer.writerow([
                        airport.get("Name", ""), airport.get("IATA", ""), airport.get("FAA", ""), airport.get("Type", ""),
                        airport.get("Latitude", ""), airport.get("Longitude", ""), airport.get("Elevation", ""),
                        airport.get("Source", ""), airport.get("State", ""), airport.get("Country", ""),
                        airport.get("Mag Declination", ""), airport.get("Use", ""), airport.get("Services", ""),
                        airport.get("Tower", ""), airport.get("Beacon", ""), runway.get("Number", ""),
                        runway.get("Length", ""), runway.get("Type", ""),
                        runway.get("Dimension", ""), runway.get("Surface", ""),
                        runway.get("Align", "")
                    ])
            else:
                writer.writerow([
                    airport.get("Name", ""), airport.get("IATA", ""), airport.get("FAA", ""), airport.get("Type", ""),
                    airport.get("Latitude", ""), airport.get("Longitude", ""), airport.get("Elevation", ""),
                    airport.get("Source", ""), airport.get("State", ""), airport.get("Country", ""),
                    airport.get("Mag Declination", ""), airport.get("Use", ""), airport.get("Services", ""),
                    airport.get("Tower", ""), airport.get("Beacon", ""), "", "", "", "", "", ""
                ])

# Main execution
if __name__ == "__main__":
    data = get_airport_data()
    if data:
        airports = parse_airport_data(data)
        save_to_csv(airports)
        print("Airport data saved to airport_info.csv")
    else:
        print("No data to save")


Airport data saved to airport_info.csv
